In [ ]:
#If you have already installed the module python-binance, please skip this step
!pip3 install python-binance
#Here is the official website if you need more information about python-binance: https://python-binance.readthedocs.io/en/latest/overview.html

In [ ]:
import numpy as np
import pandas as pd
from Grid import Grid
from binance.client import Client

In [ ]:
symbol = "BTCUSDT"
coin = "BTC"
timeFrame = "4h"
startDate = "2019-07-22"
endDate = "2022-07-22"
Binance = {"API":"MtK3YvsbesizJw7x8PzRKbcpnVDWE441aiCOjLn5qPBOX9ZLVBWCITgB8LwjBk7A", 
           "Secret":"YDGdHbK5dPkxI1noxuFLWhkE95N9jTJvwvX6uUBwAh0UhtMEzQXDhGCK2fi1aFDS"}

In [ ]:
client = Client(Binance["API"], Binance["Secret"])

In [ ]:
bars = client.get_historical_klines(symbol=f'{coin}USDT',interval=timeFrame,start_str=startDate, end_str = endDate)
test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
test_df = test_df.drop(["timestamp", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
test_df["open"] = pd.to_numeric(test_df["open"])
test_df["high"] = pd.to_numeric(test_df["high"])
test_df["low"] = pd.to_numeric(test_df["low"])
test_df["close"] = pd.to_numeric(test_df["close"])
test_df["volume"] = pd.to_numeric(test_df["volume"])

In [ ]:
test_df

In [ ]:
parameters = {
    "grid_number" : 50,
    "equal_Diff_or_Ratio" : "DIFF", 
    "lowest_price" : 3000, 
    "highest_price" : 20000,
    "start_money" : 1000,
    "trading_fee_rate" : 0.002,
    "buy_unit" : 0.0001
}

myGrid = Grid(parameters)

In [ ]:
window_size = 2000
profit = []
for i in range(len(test_df) - window_size):
    data = test_df.loc[i : i + window_size]
    data.reset_index(inplace =True)
    profit.append(myGrid.back_test_longOnly(data))

In [ ]:
expected_return = np.average(profit)
risk_free_rate = 0.0204
standard_deviation = np.std(profit)

print("Sharpe ratio:", (expected_return - risk_free_rate) / standard_deviation)